In [1]:
import numpy as np
import pandas as pd
import os
import math
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [63]:
paths = []

for i in range(1, 20):
    root, file = os.path.split("../Dataset/SA01/D01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

paths[4:14] = ['../Dataset/SA01/D05_SA01_R01_1.csv',
                '../Dataset/SA01/D05_SA01_R01_2.csv',
                '../Dataset/SA01/D05_SA01_R02_1.csv',
                '../Dataset/SA01/D05_SA01_R02_2.csv',
                '../Dataset/SA01/D05_SA01_R03_1.csv',
                '../Dataset/SA01/D05_SA01_R03_2.csv',
                '../Dataset/SA01/D05_SA01_R04_1.csv',
                '../Dataset/SA01/D05_SA01_R04_2.csv',
                '../Dataset/SA01/D05_SA01_R05_1.csv',
                '../Dataset/SA01/D05_SA01_R05_2.csv',
                '../Dataset/SA01/D06_SA01_R01_1.csv',
                '../Dataset/SA01/D06_SA01_R01_2.csv',
                '../Dataset/SA01/D06_SA01_R02_1.csv',
                '../Dataset/SA01/D06_SA01_R02_2.csv',
                '../Dataset/SA01/D06_SA01_R03_1.csv',
                '../Dataset/SA01/D06_SA01_R03_2.csv',
                '../Dataset/SA01/D06_SA01_R04_1.csv',
                '../Dataset/SA01/D06_SA01_R04_2.csv',
                '../Dataset/SA01/D06_SA01_R05_1.csv',
                '../Dataset/SA01/D06_SA01_R05_2.csv']

paths[-15:-10] = ['../Dataset/SA01/D17_SA01_R01_1.csv',
                 '../Dataset/SA01/D17_SA01_R01_2.csv',
                 '../Dataset/SA01/D17_SA01_R02_1.csv',
                 '../Dataset/SA01/D17_SA01_R02_2.csv',
                 '../Dataset/SA01/D17_SA01_R03_1.csv',
                 '../Dataset/SA01/D17_SA01_R03_2.csv',
                 '../Dataset/SA01/D17_SA01_R04_1.csv',
                 '../Dataset/SA01/D17_SA01_R04_2.csv',
                 '../Dataset/SA01/D17_SA01_R05_1.csv',
                 '../Dataset/SA01/D17_SA01_R05_2.csv']


subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23',
                'SE01', 'SE02', 'SE03', 'SE04', 'SE05',
                'SE06', 'SE07', 'SE08', 'SE09', 'SE10',
                'SE11', 'SE12', 'SE13', 'SE14', 'SE15']


total_adl_path = []

for path in paths:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_adl_path.append(path_rename)

len(total_adl_path)

['../Dataset/SE15\\D01_SE15_R01.csv',
 '../Dataset/SE15\\D02_SE15_R01.csv',
 '../Dataset/SE15\\D03_SE15_R01.csv',
 '../Dataset/SE15\\D04_SE15_R01.csv',
 '../Dataset/SE15/D05_SE15_R01_1.csv',
 '../Dataset/SE15/D05_SE15_R01_2.csv',
 '../Dataset/SE15/D05_SE15_R02_1.csv',
 '../Dataset/SE15/D05_SE15_R02_2.csv',
 '../Dataset/SE15/D05_SE15_R03_1.csv',
 '../Dataset/SE15/D05_SE15_R03_2.csv',
 '../Dataset/SE15/D05_SE15_R04_1.csv',
 '../Dataset/SE15/D05_SE15_R04_2.csv',
 '../Dataset/SE15/D05_SE15_R05_1.csv',
 '../Dataset/SE15/D05_SE15_R05_2.csv',
 '../Dataset/SE15/D06_SE15_R01_1.csv',
 '../Dataset/SE15/D06_SE15_R01_2.csv',
 '../Dataset/SE15/D06_SE15_R02_1.csv',
 '../Dataset/SE15/D06_SE15_R02_2.csv',
 '../Dataset/SE15/D06_SE15_R03_1.csv',
 '../Dataset/SE15/D06_SE15_R03_2.csv',
 '../Dataset/SE15/D06_SE15_R04_1.csv',
 '../Dataset/SE15/D06_SE15_R04_2.csv',
 '../Dataset/SE15/D06_SE15_R05_1.csv',
 '../Dataset/SE15/D06_SE15_R05_2.csv',
 '../Dataset/SE15\\D07_SE15_R01.csv',
 '../Dataset/SE15\\D07_SE15_R0

In [3]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

In [78]:
list = []
LPF_list = []
final_SE15_D = pd.DataFrame(list)  # 바꿀곳 6


for path in SE15_D_paths: # 바꿀곳 7
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        rows, cols = data.shape

        sliced_df = data[0 : 160]
    
        # LPF 구하기

        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')
    
    
        col_1_std = np.sqrt(np.absolute(col_1_lpf - np.mean(col_1_lpf))**2 / len(col_1_lpf))
        col_2_std = np.sqrt(np.absolute(col_2_lpf - np.mean(col_2_lpf))**2 / len(col_2_lpf))
        col_3_std = np.sqrt(np.absolute(col_3_lpf - np.mean(col_3_lpf))**2 / len(col_3_lpf))
    
        # 가속도 구하기
        # 표준편차 구하기

        col_1 = col_1_std*col_1_std
        col_2 = col_2_std*col_2_std
        col_3 = col_3_std*col_3_std
    
        C_nine = np.sqrt(col_1 + col_2 + col_3)
        list_C_nine = pd.DataFrame(C_nine).transpose().round(3)
        final_SE15_D = final_SE15_D.append(list_C_nine) # 바꿀곳 8

    except:
        pass

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

row, col = final_SE15_D.shape  #바꿀곳 9
target_col = row * [0]
final_SE15_D.loc[:,'target'] = target_col # 바꿀곳 10

final_SE15_D

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,target
0,18.209,18.204,18.186,18.141,18.052,17.902,17.677,17.366,16.960,16.458,15.860,15.171,14.401,13.562,12.668,11.735,10.779,9.817,8.866,7.941,7.057,6.227,5.463,4.777,4.179,3.682,3.294,3.022,2.866,2.815,2.846,2.936,3.059,3.195,3.327,3.445,3.538,3.602,3.633,3.629,3.590,3.518,3.414,3.283,3.128,2.952,2.760,2.557,2.345,2.131,...,5.881,5.682,5.453,5.212,4.981,4.785,4.652,4.603,4.649,4.786,4.997,5.255,5.532,5.804,6.047,6.247,6.392,6.474,6.491,6.445,6.341,6.187,5.993,5.770,5.531,5.288,5.053,4.839,4.655,4.509,4.403,4.337,4.304,4.295,4.296,4.296,4.281,4.239,4.164,4.048,3.890,3.688,3.446,3.170,2.869,2.555,2.245,1.960,1.727,0
0,18.308,18.302,18.278,18.218,18.098,17.896,17.591,17.167,16.614,15.927,15.107,14.163,13.107,11.958,10.738,9.475,8.202,6.956,5.785,4.750,3.935,3.440,3.324,3.535,3.937,4.407,4.868,5.276,5.612,5.864,6.032,6.117,6.124,6.060,5.932,5.748,5.516,5.244,4.941,4.619,4.291,3.974,3.691,3.464,3.320,3.273,3.327,3.464,3.659,3.881,...,6.567,6.159,5.863,5.719,5.744,5.931,6.246,6.643,7.073,7.495,7.874,8.187,8.416,8.553,8.595,8.543,8.406,8.194,7.920,7.599,7.249,6.887,6.531,6.199,5.903,5.653,5.453,5.301,5.185,5.093,5.007,4.909,4.785,4.621,4.410,4.149,3.843,3.505,3.163,2.858,2.653,2.616,2.786,3.145,3.640,4.213,4.820,5.427,6.010,0
0,18.596,18.592,18.576,18.533,18.445,18.293,18.056,17.716,17.259,16.674,15.954,15.097,14.102,12.976,11.726,10.368,8.924,7.432,5.963,4.660,3.823,3.851,4.753,6.141,7.718,9.336,10.916,12.407,13.776,14.997,16.052,16.928,17.618,18.118,18.428,18.550,18.491,18.256,17.856,17.300,16.602,15.775,14.837,13.805,12.697,11.535,10.341,9.137,7.948,6.800,...,16.993,16.356,15.600,14.739,13.787,12.761,11.673,10.542,9.382,8.212,7.053,5.928,4.870,3.925,3.170,2.714,2.653,2.948,3.452,4.043,4.650,5.236,5.787,6.294,6.758,7.178,7.561,7.909,8.229,8.527,8.807,9.074,9.330,9.576,9.812,10.036,10.243,10.430,10.591,10.723,10.820,10.876,10.884,10.838,10.735,10.573,10.359,10.112,9.861,0
0,19.039,19.034,19.016,18.970,18.877,18.721,18.483,18.151,17.713,17.164,16.502,15.730,14.854,13.886,12.838,11.725,10.567,9.383,8.192,7.016,5.877,4.798,3.808,2.944,2.265,1.855,1.773,1.949,2.239,2.543,2.810,3.021,3.170,3.259,3.293,3.278,3.222,3.135,3.023,2.897,2.762,2.625,2.491,2.365,2.249,2.145,2.053,1.972,1.903,1.843,...,1.401,1.462,1.522,1.579,1.634,1.688,1.741,1.795,1.853,1.915,1.984,2.062,2.152,2.254,2.371,2.504,2.653,2.820,3.005,3.207,3.427,3.663,3.915,4.180,4.457,4.744,5.040,5.341,5.647,5.954,6.260,6.564,6.861,7.149,7.426,7.687,7.930,8.151,8.346,8.514,8.651,8.755,8.824,8.859,8.858,8.824,8.759,8.665,8.547,0
0,18.305,18.300,18.282,18.236,18.146,17.992,17.758,17.431,17.001,16.461,15.808,15.047,14.181,13.220,12.178,11.067,9.904,8.706,7.492,6.278,5.083,3.923,2.816,1.787,0.900,0.603,1.185,1.869,2.491,3.031,3.482,3.845,4.122,4.318,4.440,4.494,4.487,4.427,4.322,4.179,4.008,3.814,3.606,3.389,3.169,2.953,2.743,2.545,2.362,2.195,...,2.002,2.024,2.049,2.076,2.104,2.132,2.160,2.187,2.211,2.233,2.251,2.264,2.273,2.277,2.276,2.271,2.261,2.248,2.232,2.214,2.194,2.173,2.152,2.132,2.113,2.096,2.081,2.068,2.057,2.048,2.042,2.037,2.034,2.032,2.032,2.032,2.032,2.032,2.032,2.032,2.032,2.032,2.032,2.032,2.034,2.036,2.039,2.044,2.050,0
0,18.355,18.351,18.333,18.287,18.196,18.041,17.804,17.473,17.034,16.482,15.813,15.031,14.140,13.149,12.072,10.924,9.720,8.481,7.225,5.973,4.748,3.574,2.493,1.599,1.178,1.497,2.150,2.835,3.470,4.027,4.500,4.887,5.190,5.413,5.563,5.646,5.670,5.644,5.574,5.469,5.335,5.179,5.004,4.817,4.620,4.415,4.207,3.995,3.783,3.572,...,1.407,1.450,1.497,1.546,1.598,1.650,1.704,1.758,1.811,1.864,1.915,1.964,2.010,2.053,2.093,2.128,2.159,2.186,2.209,2.228,2.245,2.260,2.276,2.294,2.315,2.341,2.373,2.412,2.457,2.507,2.563,2.623,2.686

In [80]:
A01 = final_SA01_D #변수지정(사람번호) - 다시해야함

In [81]:
A02 = final_SA02_D 

In [82]:
A03 = final_SA03_D

In [83]:
A04 = final_SA04_D

In [84]:
A05 = final_SA05_D

In [85]:
A06 = final_SA06_D

In [86]:
A07 = final_SA07_D

In [87]:
A08 = final_SA08_D

In [88]:
A09 = final_SA09_D

In [89]:
A10 = final_SA10_D

In [90]:
A11 = final_SA11_D

In [91]:
A12 = final_SA12_D

In [92]:
A13 = final_SA13_D

In [93]:
A14 = final_SA14_D

In [94]:
A15 = final_SA15_D

In [95]:
A16 = final_SA16_D

In [96]:
A17 = final_SA17_D

In [97]:
A18 = final_SA18_D

In [98]:
A19 = final_SA19_D

In [99]:
A20 = final_SA20_D

In [100]:
A21 = final_SA21_D

In [101]:
A22 = final_SA22_D

In [102]:
A23 = final_SA23_D

In [103]:
E01 = final_SE01_D

In [104]:
E02 = final_SE02_D

In [105]:
E03 = final_SE03_D

In [106]:
E04 = final_SE04_D

In [107]:
E05 = final_SE05_D

In [108]:
E06 = final_SE06_D

In [109]:
E07 = final_SE07_D

In [110]:
E08 = final_SE08_D

In [111]:
E09 = final_SE09_D

In [112]:
E10 = final_SE10_D

In [113]:
E11 = final_SE11_D

In [114]:
E12 = final_SE12_D

In [115]:
E13 = final_SE13_D

In [116]:
E14 = final_SE14_D

In [117]:
E15 = final_SE15_D

In [118]:
ADL_all = pd.concat([A01, A02, A03, A04, A05, A06, A07, A08, A09, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, E01, E02, E03, E04, E05, E06, E07, E08, E09, E10, E11, E12, E13, E14, E15])

In [119]:
ADL_all.to_csv('[C9]LPF_ADL.csv',index=False)